## Further Data Cleansing and Making Calls to Yelp API

- pull phone numbers from health inspection info and call Yelp to retreive relative ratings
- creatively circumvent call limits
- add findings to dataframe with intent to create visuals
- ---- this notebook starts from a CSV file of cleansed data that Anna created -----

In [25]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import matplotlib.pyplot as plt
import time

In [4]:
#This is our source file with inspection data as of March 20, 2019
file = "https://www.dropbox.com/s/99f1yjwl1xnonc8/DOHMH_New_York_City_Restaurant_Inspection_Results.csv?dl=1"

In [5]:
#Read csv file into nyc_inspect and list columns
nyc_inspect = pd.read_csv(file)
nyc_inspect.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE'],
      dtype='object')

In [6]:
#check the first 5 rows
nyc_inspect.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,50035269,FRESCO TORTILLAS,QUEENS,3112,36TH AVE,11106.0,7182679536,Tex-Mex,02/15/2019,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,9.0,A,02/15/2019,03/20/2019,Cycle Inspection / Initial Inspection
1,40861669,NEW VICTORY RESTAURANT,BROOKLYN,4803,5 AVENUE,11220.0,7184312938,Chinese,02/21/2019,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,10.0,A,02/21/2019,03/20/2019,Cycle Inspection / Re-inspection
2,50010567,CUSP,BROOKLYN,321,7TH AVE,11215.0,7187882980,CafÃ©/Coffee/Tea,03/23/2018,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,8.0,A,03/23/2018,03/20/2019,Cycle Inspection / Re-inspection
3,40821227,SABOR A MEXICO,MANHATTAN,1744,FIRST AVENUE,10128.0,2122892641,Mexican,12/08/2015,Violations were cited in the following area(s).,09C,Food contact surface not properly maintained.,Not Critical,7.0,A,12/08/2015,03/20/2019,Cycle Inspection / Initial Inspection
4,50050905,BY CHLOE,MANHATTAN,60,W 22ND ST,10010.0,6463584021,Vegetarian,12/22/2017,Violations were cited in the following area(s).,04C,Food worker does not use proper utensil to eli...,Critical,24.0,NaN,NaN,03/20/2019,Cycle Inspection / Initial Inspection


In [7]:
#count values in each column
nyc_inspect.count()

CAMIS                    385386
DBA                      384841
BORO                     385386
BUILDING                 385158
STREET                   385383
ZIPCODE                  379585
PHONE                    385379
CUISINE DESCRIPTION      385386
INSPECTION DATE          385386
ACTION                   383967
VIOLATION CODE           379351
VIOLATION DESCRIPTION    377075
CRITICAL FLAG            385386
SCORE                    366488
GRADE                    194272
GRADE DATE               192672
RECORD DATE              385386
INSPECTION TYPE          383967
dtype: int64

In [8]:
#Find out how many inspection types
nyc_inspect["INSPECTION TYPE"].value_counts()

Cycle Inspection / Initial Inspection                          222649
Cycle Inspection / Re-inspection                                93005
Pre-permit (Operational) / Initial Inspection                   26327
Pre-permit (Operational) / Re-inspection                        11960
Administrative Miscellaneous / Initial Inspection                7064
Cycle Inspection / Reopening Inspection                          4343
Pre-permit (Non-operational) / Initial Inspection                3576
Smoke-Free Air Act / Initial Inspection                          3025
Administrative Miscellaneous / Re-inspection                     2227
Trans Fat / Initial Inspection                                   2017
Pre-permit (Operational) / Compliance Inspection                 1236
Calorie Posting / Initial Inspection                             1128
Inter-Agency Task Force / Initial Inspection                     1006
Pre-permit (Operational) / Reopening Inspection                   977
Cycle Inspection / C

In [9]:
#number of occurence by "action"
rest_num_CAMIS = nyc_inspect["ACTION"].value_counts()
rest_num_CAMIS

Violations were cited in the following area(s).                                                                                        360738
Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.     13531
No violations were recorded at the time of this inspection.                                                                              4525
Establishment re-opened by DOHMH                                                                                                         3849
Establishment re-closed by DOHMH                                                                                                         1324
Name: ACTION, dtype: int64

In [10]:
#Filter out restaurant not yet in business
nyc_inspect = nyc_inspect.rename(columns={"INSPECTION DATE":"INSPECTION_DATE"})
current_nyc_inspect = nyc_inspect[nyc_inspect.INSPECTION_DATE != "01/01/1900"]
current_nyc_inspect.count()

CAMIS                    383967
DBA                      383967
BORO                     383967
BUILDING                 383745
STREET                   383967
ZIPCODE                  378186
PHONE                    383963
CUISINE DESCRIPTION      383967
INSPECTION_DATE          383967
ACTION                   383967
VIOLATION CODE           379351
VIOLATION DESCRIPTION    377075
CRITICAL FLAG            383967
SCORE                    366488
GRADE                    194272
GRADE DATE               192672
RECORD DATE              383967
INSPECTION TYPE          383967
dtype: int64

In [11]:
#filter out all inspection types other than cycle inspections
current_nyc_inspect = current_nyc_inspect.rename(columns={"INSPECTION TYPE":"INSPECTION_TYPE"})
current_nyc_cycle = current_nyc_inspect[current_nyc_inspect.INSPECTION_TYPE.str.contains('Cycle Inspection') == True]
current_nyc_cycle["INSPECTION_TYPE"].value_counts()

Cycle Inspection / Initial Inspection              222649
Cycle Inspection / Re-inspection                    93005
Cycle Inspection / Reopening Inspection              4343
Cycle Inspection / Compliance Inspection              932
Cycle Inspection / Second Compliance Inspection        28
Name: INSPECTION_TYPE, dtype: int64

In [12]:
#change values "Inspection date" column to datetime format
current_nyc_cycle['INSPECTION_DATE'] = pd.to_datetime(current_nyc_cycle['INSPECTION_DATE'])
#group data by CAMIS (unique identifer) based on latest inspection date
final_nyc_inspect = current_nyc_cycle.loc[current_nyc_cycle.groupby('CAMIS')['INSPECTION_DATE'].idxmax()]
final_nyc_inspect.head()

/anaconda3/envs/python_data/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION_DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION_TYPE
71960,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2018-05-11,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/11/2018,03/20/2019,Cycle Inspection / Initial Inspection
119570,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2019-03-04,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,13.0,A,03/04/2019,03/20/2019,Cycle Inspection / Initial Inspection
113844,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2018-05-16,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection
212732,40356018,RIVIERA CATERERS,BROOKLYN,2780,STILLWELL AVENUE,11224.0,7183723031,American,2018-10-30,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,10.0,A,10/30/2018,03/20/2019,Cycle Inspection / Initial Inspection
5948,40356151,BRUNOS ON THE BOULEVARD,QUEENS,8825,ASTORIA BOULEVARD,11369.0,7183350505,American,2018-10-06,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,45.0,NaN,NaN,03/20/2019,Cycle Inspection / Initial Inspection


In [13]:
final_nyc_inspect.to_csv("final_nyc_inspect.csv", index=False, header=True)

In [14]:
# add columns for average rating, price, and review count
final_nyc_inspect["Average Rating"] = ""
final_nyc_inspect["Price"] = ""
final_nyc_inspect["Review_Count"] = ""
final_nyc_inspect.head(-10)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION_DATE,ACTION,...,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION_TYPE,Average Rating,Price,Review_Count
71960,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2018-05-11,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/11/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
119570,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2019-03-04,Violations were cited in the following area(s).,...,Plumbing not properly installed or maintained;...,Not Critical,13.0,A,03/04/2019,03/20/2019,Cycle Inspection / Initial Inspection,,,
113844,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2018-05-16,Violations were cited in the following area(s).,...,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
212732,40356018,RIVIERA CATERERS,BROOKLYN,2780,STILLWELL AVENUE,11224.0,7183723031,American,2018-10-30,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,10.0,A,10/30/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
5948,40356151,BRUNOS ON THE BOULEVARD,QUEENS,8825,ASTORIA BOULEVARD,11369.0,7183350505,American,2018-10-06,Violations were cited in the following area(s).,...,"Raw, cooked or prepared food is adulterated, c...",Critical,45.0,NaN,NaN,03/20/2019,Cycle Inspection / Initial Inspection,,,
7404,40356483,WILKEN'S FINE FOOD,BROOKLYN,7114,AVENUE U,11234.0,7184443838,Delicatessen,2018-05-10,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,13.0,A,05/10/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
81141,40356731,TASTE THE TROPICS ICE CREAM,BROOKLYN,1839,NOSTRAND AVENUE,11226.0,7188560821,"Ice Cream, Gelato, Yogurt, Ices",2018-05-16,Violations were cited in the following area(s).,...,Pesticide use not in accordance with label or ...,Not Critical,13.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
246415,40357217,WILD ASIA,BRONX,2300,SOUTHERN BOULEVARD,10460.0,7182207846,American,2018-05-31,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/31/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
92248,40359480,1 EAST 66TH STREET KITCHEN,MANHATTAN,1,EAST 66 STREET,10065.0,2128793900,American,2018-09-28,Violations were cited in the following area(s).,...,"Food contact surface not properly washed, rins...",Critical,7.0,A,09/28/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
237718,40359705,NATHAN'S FAMOUS,BROOKLYN,1310,SURF AVENUE,11224.0,7183332202,Hotdogs,2018-03-07,Violations were cited in the following area(s).,...,Food not protected from potential source of co...,Critical,10.0,A,03/07/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,


In [24]:
nyc_f_i = final_nyc_inspect.reset_index(drop=True)
nyc_f_i.head()


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION_DATE,ACTION,...,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION_TYPE,Average Rating,Price,Review_Count
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2018-05-11,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/11/2018,03/20/2019,Cycle Inspection / Initial Inspection,4,$$,41
1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2019-03-04,Violations were cited in the following area(s).,...,Plumbing not properly installed or maintained;...,Not Critical,13.0,A,03/04/2019,03/20/2019,Cycle Inspection / Initial Inspection,2,$,36
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2018-05-16,Violations were cited in the following area(s).,...,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,3.5,$$,99
3,40356018,RIVIERA CATERERS,BROOKLYN,2780,STILLWELL AVENUE,11224.0,7183723031,American,2018-10-30,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,10.0,A,10/30/2018,03/20/2019,Cycle Inspection / Initial Inspection,4,,
4,40356151,BRUNOS ON THE BOULEVARD,QUEENS,8825,ASTORIA BOULEVARD,11369.0,7183350505,American,2018-10-06,Violations were cited in the following area(s).,...,"Raw, cooked or prepared food is adulterated, c...",Critical,45.0,NaN,NaN,03/20/2019,Cycle Inspection / Initial Inspection,4,,


In [29]:
# sshhh.. call yelp api using different api keys so we could get all the data we needed in a hurry- 
# the limit is 5000/day. not quite enough to get things done in time..

api_key = ""
api_key1 = ""
api_key2 = ""
api_key3 = ""


e_count = 0
phone_count = 0
nophone_count = 0
index = 0
call = 0

for index, row in nyc_f_i.iterrows():
    time.sleep(.5)
    call += 1
    try:
        phone_count += 1
                
        phone1 = row["PHONE"]
        phone = str(phone1)
        url = 'https://api.yelp.com/v3/businesses/search/phone'
        headers = {'Authorization': 'Bearer {}'.format(api_key),}
        url_params = {'phone': '+1'+ phone,}
        response = requests.get(url, headers=headers, params=url_params)

        avg_rating = response.json()

        nyc_f_i.loc[index, "Average Rating"] = avg_rating["businesses"][0]['rating']
        nyc_f_i.loc[index, "Price"] = avg_rating["businesses"][0]['price']
        nyc_f_i.loc[index, "Review_Count"] = avg_rating["businesses"][0]['review_count']
        
        if (phone_count) % 100 == 0:
            print(phone_count)
            print(api_key)
            
        # switch keys 
        if call >= 5000 and call < 10000:           
            api_key = api_key1
                        
        if call >= 10000 and call < 15000:
            api_key = api_key2
                       
        if call >= 15000:
            api_key = api_key3
            
        if call >= 20000:
            break
            
            
    # print exceptions         
    except Exception as e:
        
       
        e_count += 1
        if (e_count) % 100 == 0:            
            print(f"error number {e_count}")
            print(e) 
            print(phone)

100
oLU1rLUWaOUhWm7WyEs41h6Wsrsrd_0Xlg6HbDSUAG71heTuQp3zH8cGpXfZRJCe-x85DBrYCOly7ck0nD_AI3YQU7XoPuXeiyiAJP9SwAXN7wZP8lcSDbqtwaybXHYx
200
oLU1rLUWaOUhWm7WyEs41h6Wsrsrd_0Xlg6HbDSUAG71heTuQp3zH8cGpXfZRJCe-x85DBrYCOly7ck0nD_AI3YQU7XoPuXeiyiAJP9SwAXN7wZP8lcSDbqtwaybXHYx
300
oLU1rLUWaOUhWm7WyEs41h6Wsrsrd_0Xlg6HbDSUAG71heTuQp3zH8cGpXfZRJCe-x85DBrYCOly7ck0nD_AI3YQU7XoPuXeiyiAJP9SwAXN7wZP8lcSDbqtwaybXHYx
400
oLU1rLUWaOUhWm7WyEs41h6Wsrsrd_0Xlg6HbDSUAG71heTuQp3zH8cGpXfZRJCe-x85DBrYCOly7ck0nD_AI3YQU7XoPuXeiyiAJP9SwAXN7wZP8lcSDbqtwaybXHYx
500
oLU1rLUWaOUhWm7WyEs41h6Wsrsrd_0Xlg6HbDSUAG71heTuQp3zH8cGpXfZRJCe-x85DBrYCOly7ck0nD_AI3YQU7XoPuXeiyiAJP9SwAXN7wZP8lcSDbqtwaybXHYx
error number 100
list index out of range
7185965954
600
oLU1rLUWaOUhWm7WyEs41h6Wsrsrd_0Xlg6HbDSUAG71heTuQp3zH8cGpXfZRJCe-x85DBrYCOly7ck0nD_AI3YQU7XoPuXeiyiAJP9SwAXN7wZP8lcSDbqtwaybXHYx
700
oLU1rLUWaOUhWm7WyEs41h6Wsrsrd_0Xlg6HbDSUAG71heTuQp3zH8cGpXfZRJCe-x85DBrYCOly7ck0nD_AI3YQU7XoPuXeiyiAJP9SwAXN7wZP8lcSDbqtwaybXHYx
800
oLU1rLUWaOUhW

7700
ozHhOtWw-5Q34rvXhzl4L5GUeXP2L7cKZIvxO4bOe8En5I0aZEhJ-mo4Jb-visg0lcNUIgPNuUO1lvQZjanSaQQ_rRzXShvF6hZwEt3_330j58T73-ZYNaBGd46bXHYx
7800
ozHhOtWw-5Q34rvXhzl4L5GUeXP2L7cKZIvxO4bOe8En5I0aZEhJ-mo4Jb-visg0lcNUIgPNuUO1lvQZjanSaQQ_rRzXShvF6hZwEt3_330j58T73-ZYNaBGd46bXHYx
7900
ozHhOtWw-5Q34rvXhzl4L5GUeXP2L7cKZIvxO4bOe8En5I0aZEhJ-mo4Jb-visg0lcNUIgPNuUO1lvQZjanSaQQ_rRzXShvF6hZwEt3_330j58T73-ZYNaBGd46bXHYx
error number 1900
'price'
2125050886
8000
ozHhOtWw-5Q34rvXhzl4L5GUeXP2L7cKZIvxO4bOe8En5I0aZEhJ-mo4Jb-visg0lcNUIgPNuUO1lvQZjanSaQQ_rRzXShvF6hZwEt3_330j58T73-ZYNaBGd46bXHYx
8200
ozHhOtWw-5Q34rvXhzl4L5GUeXP2L7cKZIvxO4bOe8En5I0aZEhJ-mo4Jb-visg0lcNUIgPNuUO1lvQZjanSaQQ_rRzXShvF6hZwEt3_330j58T73-ZYNaBGd46bXHYx
error number 2000
list index out of range
2122198890
8300
ozHhOtWw-5Q34rvXhzl4L5GUeXP2L7cKZIvxO4bOe8En5I0aZEhJ-mo4Jb-visg0lcNUIgPNuUO1lvQZjanSaQQ_rRzXShvF6hZwEt3_330j58T73-ZYNaBGd46bXHYx
8500
ozHhOtWw-5Q34rvXhzl4L5GUeXP2L7cKZIvxO4bOe8En5I0aZEhJ-mo4Jb-visg0lcNUIgPNuUO1lvQZjanSaQQ_rRzXShvF6hZwE

15200
n8YTDyutMz9YxhPn156OoCUQQjzqUbq2Frty5VkvrMVTGvFAdwY-06wJ6_me5lr5ZyTgp1X1f-1m6_6uyDws6BvLr9uwKQp_QvtHGFm37XX2-F5UXanSlVipEd2bXHYx
15300
n8YTDyutMz9YxhPn156OoCUQQjzqUbq2Frty5VkvrMVTGvFAdwY-06wJ6_me5lr5ZyTgp1X1f-1m6_6uyDws6BvLr9uwKQp_QvtHGFm37XX2-F5UXanSlVipEd2bXHYx
error number 4200
list index out of range
9724444957
15400
n8YTDyutMz9YxhPn156OoCUQQjzqUbq2Frty5VkvrMVTGvFAdwY-06wJ6_me5lr5ZyTgp1X1f-1m6_6uyDws6BvLr9uwKQp_QvtHGFm37XX2-F5UXanSlVipEd2bXHYx
15500
n8YTDyutMz9YxhPn156OoCUQQjzqUbq2Frty5VkvrMVTGvFAdwY-06wJ6_me5lr5ZyTgp1X1f-1m6_6uyDws6BvLr9uwKQp_QvtHGFm37XX2-F5UXanSlVipEd2bXHYx
error number 4300
list index out of range
7188961836
15600
n8YTDyutMz9YxhPn156OoCUQQjzqUbq2Frty5VkvrMVTGvFAdwY-06wJ6_me5lr5ZyTgp1X1f-1m6_6uyDws6BvLr9uwKQp_QvtHGFm37XX2-F5UXanSlVipEd2bXHYx
15700
n8YTDyutMz9YxhPn156OoCUQQjzqUbq2Frty5VkvrMVTGvFAdwY-06wJ6_me5lr5ZyTgp1X1f-1m6_6uyDws6BvLr9uwKQp_QvtHGFm37XX2-F5UXanSlVipEd2bXHYx
error number 4400
list index out of range
9175538898
15900
n8YTDyutMz9YxhPn156OoCUQQ

In [31]:
nyc_f_i.head(20000)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION_DATE,ACTION,...,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION_TYPE,Average Rating,Price,Review_Count
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2018-05-11,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/11/2018,03/20/2019,Cycle Inspection / Initial Inspection,4,$$,41
1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2019-03-04,Violations were cited in the following area(s).,...,Plumbing not properly installed or maintained;...,Not Critical,13.0,A,03/04/2019,03/20/2019,Cycle Inspection / Initial Inspection,2,$,36
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2018-05-16,Violations were cited in the following area(s).,...,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,3.5,$$,99
3,40356018,RIVIERA CATERERS,BROOKLYN,2780,STILLWELL AVENUE,11224.0,7183723031,American,2018-10-30,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,10.0,A,10/30/2018,03/20/2019,Cycle Inspection / Initial Inspection,4,,
4,40356151,BRUNOS ON THE BOULEVARD,QUEENS,8825,ASTORIA BOULEVARD,11369.0,7183350505,American,2018-10-06,Violations were cited in the following area(s).,...,"Raw, cooked or prepared food is adulterated, c...",Critical,45.0,NaN,NaN,03/20/2019,Cycle Inspection / Initial Inspection,4,,
5,40356483,WILKEN'S FINE FOOD,BROOKLYN,7114,AVENUE U,11234.0,7184443838,Delicatessen,2018-05-10,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,13.0,A,05/10/2018,03/20/2019,Cycle Inspection / Initial Inspection,3.5,$$,37
6,40356731,TASTE THE TROPICS ICE CREAM,BROOKLYN,1839,NOSTRAND AVENUE,11226.0,7188560821,"Ice Cream, Gelato, Yogurt, Ices",2018-05-16,Violations were cited in the following area(s).,...,Pesticide use not in accordance with label or ...,Not Critical,13.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,4.5,$,31
7,40357217,WILD ASIA,BRONX,2300,SOUTHERN BOULEVARD,10460.0,7182207846,American,2018-05-31,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/31/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
8,40359480,1 EAST 66TH STREET KITCHEN,MANHATTAN,1,EAST 66 STREET,10065.0,2128793900,American,2018-09-28,Violations were cited in the following area(s).,...,"Food contact surface not properly washed, rins...",Critical,7.0,A,09/28/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,
9,40359705,NATHAN'S FAMOUS,BROOKLYN,1310,SURF AVENUE,11224.0,7183332202,Hotdogs,2018-03-07,Violations were cited in the following area(s).,...,Food not protected from potential source of co...,Critical,10.0,A,03/07/2018,03/20/2019,Cycle Inspection / Initial Inspection,,,


In [35]:
# remove businesses that were not found on yelp api
nyc_f_i_20k = nyc_f_i[(nyc_f_i['Average Rating'].str.len() != 0)]

nyc_f_i_20k.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION_DATE,ACTION,...,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION_TYPE,Average Rating,Price,Review_Count
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2018-05-11,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/11/2018,03/20/2019,Cycle Inspection / Initial Inspection,4,$$,41
1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2019-03-04,Violations were cited in the following area(s).,...,Plumbing not properly installed or maintained;...,Not Critical,13.0,A,03/04/2019,03/20/2019,Cycle Inspection / Initial Inspection,2,$,36
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2018-05-16,Violations were cited in the following area(s).,...,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,3.5,$$,99
3,40356018,RIVIERA CATERERS,BROOKLYN,2780,STILLWELL AVENUE,11224.0,7183723031,American,2018-10-30,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,10.0,A,10/30/2018,03/20/2019,Cycle Inspection / Initial Inspection,4,,
4,40356151,BRUNOS ON THE BOULEVARD,QUEENS,8825,ASTORIA BOULEVARD,11369.0,7183350505,American,2018-10-06,Violations were cited in the following area(s).,...,"Raw, cooked or prepared food is adulterated, c...",Critical,45.0,NaN,NaN,03/20/2019,Cycle Inspection / Initial Inspection,4,,


In [36]:
nyc_f_i_20k.to_csv("nyc_f_i-20k.csv", index=False, header=True)

In [37]:
# convert column strings to values
nyc_f_i_20k["SCORE"] = nyc_f_i_20k["SCORE"].apply(pd.to_numeric, errors='coerce')
nyc_f_i_20k["Average Rating"] = nyc_f_i_20k["Average Rating"].apply(pd.to_numeric, errors='coerce')

/anaconda3/envs/python_data/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/python_data/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
# remove businesses that did not have a price rating on yelp
final_df = nyc_f_i_20k[(nyc_f_i_20k['Price'].str.len() != 0)]
final_df.to_csv("nyc_f_i_20k_plus.csv", index=False, header=True)
final_df.head(20000)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION_DATE,ACTION,...,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION_TYPE,Average Rating,Price,Review_Count
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2018-05-11,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/11/2018,03/20/2019,Cycle Inspection / Initial Inspection,4.0,$$,41
1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2019-03-04,Violations were cited in the following area(s).,...,Plumbing not properly installed or maintained;...,Not Critical,13.0,A,03/04/2019,03/20/2019,Cycle Inspection / Initial Inspection,2.0,$,36
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2018-05-16,Violations were cited in the following area(s).,...,Facility not vermin proof. Harborage or condit...,Not Critical,12.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,3.5,$$,99
5,40356483,WILKEN'S FINE FOOD,BROOKLYN,7114,AVENUE U,11234.0,7184443838,Delicatessen,2018-05-10,Violations were cited in the following area(s).,...,Non-food contact surface improperly constructe...,Not Critical,13.0,A,05/10/2018,03/20/2019,Cycle Inspection / Initial Inspection,3.5,$$,37
6,40356731,TASTE THE TROPICS ICE CREAM,BROOKLYN,1839,NOSTRAND AVENUE,11226.0,7188560821,"Ice Cream, Gelato, Yogurt, Ices",2018-05-16,Violations were cited in the following area(s).,...,Pesticide use not in accordance with label or ...,Not Critical,13.0,A,05/16/2018,03/20/2019,Cycle Inspection / Initial Inspection,4.5,$,31
11,40360076,CARVEL ICE CREAM,BROOKLYN,203,CHURCH AVENUE,11218.0,7184389501,"Ice Cream, Gelato, Yogurt, Ices",2018-03-20,Violations were cited in the following area(s).,...,Facility not vermin proof. Harborage or condit...,Not Critical,13.0,A,03/20/2018,03/20/2019,Cycle Inspection / Initial Inspection,3.5,$,27
12,40361322,CARVEL ICE CREAM,QUEENS,26515,HILLSIDE AVENUE,11004.0,7183430392,"Ice Cream, Gelato, Yogurt, Ices",2018-07-12,Violations were cited in the following area(s).,...,Food Protection Certificate not held by superv...,Critical,25.0,B,07/12/2018,03/20/2019,Cycle Inspection / Re-inspection,4.5,$$,3
13,40361618,SAL'S DELI,QUEENS,12908,20 AVENUE,11356.0,7186619498,Delicatessen,2018-06-05,Violations were cited in the following area(s).,...,Food not protected from potential source of co...,Critical,11.0,A,06/05/2018,03/20/2019,Cycle Inspection / Initial Inspection,4.5,$,10
14,40361998,STEVE CHU'S DELI & GROCERY,QUEENS,3406,10 STREET,11106.0,7187268888,Delicatessen,2018-07-18,Violations were cited in the following area(s).,...,Wiping cloths soiled or not stored in sanitizi...,Critical,5.0,A,07/18/2018,03/20/2019,Cycle Inspection / Initial Inspection,4.0,$,4
15,40362098,HARRIET'S KITCHEN,MANHATTAN,502,AMSTERDAM AVENUE,10024.0,2127210045,American,2018-11-28,Violations were cited in the following area(s).,...,"Raw, cooked or prepared food is adulterated, c...",Critical,19.0,B,11/28/2018,03/20/2019,Cycle Inspection / Re-inspection,3.0,$$,142
